In [14]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

## Define dataset and variables

In [24]:
case = "fanv3_debugging"
path = "/glade/scratch/jinmuluo/archive/one_year/lnd/hist/" 
start_date = date(2000, 1, 1)
end_date = date(2000, 12, 1)
delta = end_date - start_date

In [17]:
fieldsh2 = ["N2O_DENITRIFY_TOTAL", "N2O_NITRIFY_TOTAL", "NOx_DENITRIFY_TOTAL", "NOx_NITRIFY_TOTAL", "area"]
fieldsh1 = ["NITRITE_G1", "NITRITE_G2", "NITRITE_G3", "NITRITE_S0", "NITRITE_S1", "NITRITE_S2",
            "NITRITE_S3", "NITRITE_F1", "NITRITE_F2",  "NITRITE_F3",  "NITRITE_F4",'area']
fieldsh0 = ["GPP", "FERT_NO3_TO_SOIL", "MANURE_NO3_TO_SOIL", "MANURE_NITRITE_RUNOFF", 'FERT_NITRITE_RUNOFF', 
            "FERT_NITRITE_TO_SOIL", "MANURE_NITRITE_TO_SOIL", 'area']

## Nitrification flux from fanv2

In [12]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")

In [16]:
def preprocess(ds, fields = fieldsh0):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    
    return ds

dsCLM = fix_time(xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess))
# dsCLM

In [70]:
ds = (dsCLM["FERT_NO3_TO_SOIL"] + dsCLM["MANURE_NO3_TO_SOIL"]) * dsCLM['area']*1e6
ds = ds.sum(dim=['lat', 'lon'])
fanv2_flux = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    fanv2_flux = fanv2_flux + ds[i].values * day * 24 * 3600
fanv2_flux = fanv2_flux * 1e-12

print(fanv2_flux, "Tg/year")

43.7996411694 Tg/year


## NOX and N2O emission 

In [71]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h2" + "." + month +".nc")

In [73]:
def preprocess(ds, fields = fieldsh2):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    
    return ds

dsCLM = fix_time(xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess))
# dsCLM

In [83]:
ds_n2o = (dsCLM["N2O_DENITRIFY_TOTAL"] + dsCLM["N2O_NITRIFY_TOTAL"]) * dsCLM['area']*1e6
ds_nox = (dsCLM["NOx_DENITRIFY_TOTAL"] + dsCLM["NOx_NITRIFY_TOTAL"]) * dsCLM['area']*1e6

ds_n2o = ds_n2o.sum(dim=['lat', 'lon'])
ds_nox = ds_nox.sum(dim=['lat', 'lon'])

n2o_total = 0.0
nox_total = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    n2o_total = n2o_total + ds_n2o[i].values * day * 24 * 3600
    nox_total = nox_total + ds_nox[i].values * day * 24 * 3600
    
n2o_total = n2o_total * 1e-12
nox_total = nox_total * 1e-12

print('N2O:', n2o_total, "Tg/year")
print('NOx:', nox_total, "Tg/year")

N2O: 0.635161109765625 Tg/year
NOx: 0.1499394211734375 Tg/year


## Diffusion and Runoff